In [7]:
%display latex

In [8]:
sage: maxima_calculus.eval('domain:real')
'real'

'real'

Define the manifold $M$ with coordinates $(t, r, \theta, \phi)$ and metric $g_{a' b'} = diag(-1, 1, r^2, r^2 \sin^2 \theta)$, and set $\theta = \pi/2$. Compute Christoffel symbols for this metric.

In [9]:
M = Manifold(4, 'M', structure='Lorentzian')
X.<t, r, th, ph> = M.chart(r't r:(0,+oo) th:(0,pi):\theta ph:(0,2*pi):\phi')
g = M.metric('g')
g[0, 0], g[1, 1], g[2, 2], g[3, 3] = -1, 1, r^2, r^2*sin(th)^2
g[:]

[           -1             0             0             0]
[            0             1             0             0]
[            0             0           r^2             0]
[            0             0             0 r^2*sin(th)^2]

In [10]:
g.apply_map(lambda x: x.subs({th:pi/2}))
g[:]

[ -1   0   0   0]
[  0   1   0   0]
[  0   0 r^2   0]
[  0   0   0 r^2]

In [11]:
Gam = g.christoffel_symbols()
nab = M.affine_connection('nabla', r'\nabla')
nab[:] = Gam[:]
nab[:]

[[[0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0]],
 [[0, 0, 0, 0], [0, 0, 0, 0], [0, 0, -r, 0], [0, 0, 0, -r]],
 [[0, 0, 0, 0], [0, 0, 1/r, 0], [0, 1/r, 0, 0], [0, 0, 0, 0]],
 [[0, 0, 0, 0], [0, 0, 0, 1/r], [0, 0, 0, 0], [0, 1/r, 0, 0]]]

In [12]:
nab.display()

Gam^r_th,th = -r 
Gam^r_ph,ph = -r 
Gam^th_r,th = 1/r 
Gam^th_th,r = 1/r 
Gam^ph_r,ph = 1/r 
Gam^ph_ph,r = 1/r

Define the variables $s, q, Q, m$, as respectively the parameter along the curves, the test charge, the source charge, and the test mass. Also define the variables $\epsilon, \kappa, \Phi_0$ as respectively the ultrarelativistic parameter, the angular momentum proportionality factor $l = \kappa/\epsilon$, and the initial azimuthal angle. Also define the ``outside-in'' functions $(U, V, X^1, X^2)$ which at $\epsilon = 0$ become the PW coordinates. 

In [13]:
s, q, Q, m = var('s, q, Q, m')
eps, kap, phi_init = var('epsilon, kappa, Phi_0')
assume(eps>=0)
assume(s>=0)
assume(kap>0)
assume(m>0)
s, eps, kap

(s, epsilon, kappa)

In [14]:
U = function('U', nargs=2)(s, eps)
V = function('V', nargs=2)(s, eps)
X1 = function('X_1', nargs=2)(s, eps)
X2 = function('X_2', nargs=2)(s, eps)
U, V, X1, X2

(U(s, epsilon), V(s, epsilon), X_1(s, epsilon), X_2(s, epsilon))

Define the tangent and deviation vectors respectively $\partial_U Y^{a'}$ and $\xi^{a'} \equiv \partial_{\epsilon} Y^{a'}$. The tangent vector has components $\partial_U Y^{a'} = (\partial_U t, \partial_U r, 0, \partial_U \phi)$ where $\partial_U t = \frac{1}{m} \bigg ( 1 + \epsilon \frac{q Q}{r} \bigg ), \\
\partial_U r = \frac{1}{m} \sqrt{1 - \frac{\kappa^2}{r^2} + \epsilon \frac{2 q Q}{r} + \epsilon^2 \bigg ( \frac{q^2 Q^2}{r^2} - m^2 \bigg )}, \\
\partial_U \phi = \frac{\kappa}{m r^2}$.

and the deviation vector has components $\xi^{a'} = (\xi^t, \xi^r, \xi^{\theta}, \xi^{\phi}) = (\partial_{\epsilon} t, \partial_{\epsilon} r, 0, \partial_{\epsilon} \phi)$. Note that since $\theta = \pi/2$ is fixed, $\xi^{\theta} = \partial_{\epsilon} \theta = 0$.

In [15]:
dsY = M.vector_field(name=r'\partial_s Y')
xi = M.vector_field(name=r'\xi')
dsY, xi

(Vector field \partial_s Y on the 4-dimensional Lorentzian manifold M,
 Vector field \xi on the 4-dimensional Lorentzian manifold M)

In [16]:
ds_t = 1/m*(1 + eps*q*Q/r)
ds_r = 1/m*sqrt(1 - kap^2/r^2 + eps*2*q*Q/r + eps^2*(q^2*Q^2/r^2 - m^2))
ds_ph = kap/(m * r^2)
ds_t, ds_r, ds_ph

((Q*epsilon*q/r + 1)/m,
 sqrt(-(m^2 - Q^2*q^2/r^2)*epsilon^2 + 2*Q*epsilon*q/r - kappa^2/r^2 + 1)/m,
 kappa/(m*r^2))

In [17]:
xi_t = function('xi_t')(s, eps)
xi_r = function('xi_r')(s, eps)
xi_ph = function('xi_ph')(s, eps)
dsY[:] = [ds_t, ds_r, 0, ds_ph]
xi[:] = [xi_t, xi_r, 0, xi_ph]
dsY[:], xi[:]

([(Q*epsilon*q/r + 1)/m,
  sqrt(-(m^2 - Q^2*q^2/r^2)*epsilon^2 + 2*Q*epsilon*q/r - kappa^2/r^2 + 1)/m,
  0,
  kappa/(m*r^2)],
 [xi_t(s, epsilon), xi_r(s, epsilon), 0, xi_ph(s, epsilon)])

The leading order solution (the equation of the reference null geodesic) can be obtained by setting $\epsilon = 0$ and integrating the components of the tangent vector.

In [18]:
ds_Y0 = dsY.copy()
ds_Y0.apply_map(lambda x: x.subs({eps:0}))
ds_Y0[:], dsY[:]

([1/m, sqrt(-kappa^2/r^2 + 1)/m, 0, kappa/(m*r^2)],
 [(Q*epsilon*q/r + 1)/m,
  sqrt(-(m^2 - Q^2*q^2/r^2)*epsilon^2 + 2*Q*epsilon*q/r - kappa^2/r^2 + 1)/m,
  0,
  kappa/(m*r^2)])

In [19]:
t0_sol = ds_Y0[0].expr().integrate(s)
t0_sol

s/m

To integrate the $r$ EoM, introduce the new function $R(s) = r(s, \epsilon = 0)$, the differential equation then becomes an ODE as shown below.

In [20]:
R = function('R')(s)
assume(R>0)
dR = R.diff(s, 1)
dR_rhs = ds_Y0[1].expr().subs({r:R})
ode_r0 = dR == dR_rhs
ode_r0

diff(R(s), s) == sqrt(-kappa^2/R(s)^2 + 1)/m

In [21]:
r0_tempsol = desolve(ode_r0, R, ivar=s)
r0_tempsol

m*integrate(R(s)/sqrt(-kappa^2 + R(s)^2), R(s)) == _C + s

In [22]:
r0_tempsol.lhs().simplify()

sqrt(-kappa^2 + R(s)^2)*m

We can now set the integration constant to vanish by choosing $R(s = 0) = \kappa$. Thus $r (s, \epsilon = 0) = R (s) = \sqrt{\kappa^2 + s^2/m^2}$

In [23]:
r0_sol = sqrt(kap^2 + s^2/m^2)
r0_sol

sqrt(kappa^2 + s^2/m^2)

In [24]:
ph0_tempsol = ds_Y0[3].expr().subs({r:r0_sol}).integrate(s)
ph0_tempsol

arctan(s/(kappa*m))

Keeping in mind that the initial azimuthal angle is $\phi (s = 0, \epsilon = 0) = \Phi_0$, we need to add this as the integration constant to our existing solution.

In [25]:
ph0_sol = ph0_tempsol + phi_init
ph0_sol

Phi_0 + arctan(s/(kappa*m))

The leading order solution (essentially the reference null geodesic) is thus described by
$z (s) = \bigg ( \frac{s}{m}, \sqrt{\kappa^2 + \frac{s^2}{m^2}}, \frac{\pi}{2}, \arctan{\big ( \frac{s}{\kappa m} \big ) + \Phi_0 \bigg )}$.

In [26]:
z_ng = M.curve({X: [t0_sol, r0_sol, pi/2, ph0_sol]}, (s, 0, oo), name='z')
z_ng.display()

z: (0, +Infinity) → M
   s ↦ (t, r, th, ph) = (s/m, sqrt(kappa^2 + s^2/m^2), 1/2*pi, Phi_0 + arctan(s/(kappa*m)))

Define the parallel propagated tetrad $(\dot{z}, n, e_1, e_2)$ on the null geodesic. See the sage file ``Flat space Coulomb tetrad'' for details on computing the form of the tetrad. Check that these vectors satisfy the quasi null orthogonality relations as well the parallel transport condition (that their acceleration vanishes).

In [27]:
zdot = M.vector_field(name=r'\dot{z}')
n = M.vector_field(name='n')
e1 = M.vector_field(name='e_1')
e2 = M.vector_field(name='e_2')
zdot[:] = [1/m, 1/m*sqrt(1 - kap^2/r^2), 0, kap/(m*r^2)]
n[:] = [m/2, -m/2*sqrt(1 - kap^2/r^2), 0, -kap*m/(2*r^2)]
e1[:] = [0, 0, 1/r, 0]
e2[:] = [0, -kap/r, 0, 1/r*sqrt(1 - kap^2/r^2)]
zdot[:], n[:], e1[:], e2[:]

([1/m, sqrt(-kappa^2/r^2 + 1)/m, 0, kappa/(m*r^2)],
 [1/2*m, -1/2*m*sqrt(-kappa^2/r^2 + 1), 0, -1/2*kappa*m/r^2],
 [0, 0, 1/r, 0],
 [0, -kappa/r, 0, sqrt(-kappa^2/r^2 + 1)/r])

In [28]:
g(zdot, zdot).display(), g(zdot, n).display(), g(zdot, e1).display(), g(zdot, e2).display()

(g(\dot{z},\dot{z}): M → ℝ
    (t, r, th, ph) ↦ 0,
 g(\dot{z},n): M → ℝ
    (t, r, th, ph) ↦ -1,
 g(\dot{z},e_1): M → ℝ
    (t, r, th, ph) ↦ 0,
 g(\dot{z},e_2): M → ℝ
    (t, r, th, ph) ↦ 0)

In [29]:
g(n, n).display(), g(n, e1).display(), g(n, e2).display()

(g(n,n): M → ℝ
    (t, r, th, ph) ↦ 0,
 g(n,e_1): M → ℝ
    (t, r, th, ph) ↦ 0,
 g(n,e_2): M → ℝ
    (t, r, th, ph) ↦ 0)

In [30]:
g(e1, e1).display(), g(e1, e2).display(), g(e2, e2).display()

(g(e_1,e_1): M → ℝ
    (t, r, th, ph) ↦ 1,
 g(e_1,e_2): M → ℝ
    (t, r, th, ph) ↦ 0,
 g(e_2,e_2): M → ℝ
    (t, r, th, ph) ↦ 1)

In [31]:
acc_z = nab(zdot).contract(zdot)
acc_n = nab(n).contract(zdot)
acc_e1 = nab(e1).contract(zdot)
acc_e2 = nab(e2).contract(zdot)
acc_z[:], acc_n[:], acc_e1[:], acc_e2[:]

([0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0])

Using the outside-in approach, the deviation vector at coincidence has the form $[\xi^{a'}] = \dot{z}^a [\partial_{\epsilon} U] + e^a_i [X^i]$. But from our above analysis, we know that the $\theta$ component of this vanishes. Thus $[\xi^{\theta}] = \dot{z}^{\theta} [\partial_{\epsilon} U] + e^{\theta}_i [X^i] = 0$, from which it follows that $[X^1] = 0$.

In [32]:
depsU0 = U.diff(eps, 1).subs({eps:0})
dsU0 = U.diff(s, 1).subs({eps:0})
X1_0 = X1.subs({eps:0})
X2_0 = X2.subs({eps:0})
dsV0 = V.diff(s, 1).subs({eps:0})
V0 = V.subs({eps:0})
depsU0, dsU0, X1_0, X2_0, V0, dsV0

(D[1](U)(s, 0), D[0](U)(s, 0), X_1(s, 0), X_2(s, 0), V(s, 0), D[0](V)(s, 0))

In [33]:
xi_CL_out_in = zdot*depsU0 + e1*X1_0 + e2*X2_0
xi_CL_out_in[:]

[d(U)/d0/m,
 -(kappa*m*X_2(s, 0) - sqrt(kappa + r)*sqrt(-kappa + r)*d(U)/d0)/(m*r),
 X_1(s, 0)/r,
 (sqrt(kappa + r)*sqrt(-kappa + r)*m*X_2(s, 0) + kappa*d(U)/d0)/(m*r^2)]

Now the covariant $\epsilon$-derivative of the tangent vector $\partial_s Y$ calculated in the outside-in approach has the coincidence limit $[D_{\epsilon s} Y^{a'}] = \dot{z}^a [\partial_{\epsilon s} U] + e^a_i [\partial_s X^i]$. Furthermore, since we know the form of the tangent vector explicitly (in terms of the coordinates and the parameters), we can compute the covariant derivatives through the standard algorithm $D_{\epsilon} \partial_s Y^{a'} = \xi^{b'} \nabla_{b'} (\partial_s Y^{a'}) = \partial_{\epsilon} (\partial_s Y^{a'}) + \Gamma^{a'}{}_{b' c'} \partial_s Y^{c'} \xi^{b'}$. We can compute the covariant derivatives and plug the result into the LHS of the outside-in expression mentioned above to arrive at a system of ODEs involving the plane wave coordinate functions. By equating the two equations, we get
$[ \partial_{\epsilon} (\partial_s Y^{a'}) + \Gamma^{a'}{}_{b' c'} \partial_s Y^{c'} \xi^{b'} ] = \dot{z}^a [\partial_{\epsilon s} U] + e^a_i [\partial_s X^i]$.

Below, we first compute the covariant deriavtives, and then derive the relevant expressions for the outside-in approach.

Define a function that computes the covariant $\epsilon$-derivative of the input vectors. Note the following: 
1. the input vector depends on $\epsilon$ through scalar factors as well as implicitly in the coordinates, for e.g. the $t$-component of the tangent vector $\partial_U t = \frac{1}{m} \bigg ( 1 + \epsilon \frac{q Q}{r} \bigg )$ contains $\epsilon$ explicitly as well as implicitly (through the coordinate $r$).

2. Sage computes covariant derivatives through coordinate derivatives, where the coordinates are independent of parameters. Essentially, Sage computes $\nabla_{b'} \chi^{a'} = \partial_{b'} \chi^{a'} + \Gamma^{a'}{}_{b' c'} \chi^{c'}$. If we naively use the idea that $D_{\epsilon} = \xi^{b'} \nabla_{b'}$, then we miss differentiating scalar factors of $\epsilon$ that might appear in the components of $\chi^{a'}$.

Thus to account for scalar factors of $\epsilon$, we compute these separately and add them to the covariant derivative computed by Sage. Thus we consider that the coordinates $Y^{a'}$ are independent of the parameter $\epsilon$ and then
$D_{\epsilon} \chi^{a'} = \partial_{\epsilon} \chi^{a'} + \xi^{b'} \nabla_{b'} \chi^{a'}$, where the first term only differentiates explicit factors of $\epsilon$.

In [34]:
def D_eps(inp_vec):
    D_eps_vec = M.vector_field(name=r'D_{\epsilon}V')
    for i in range(4):
        D_eps_vec[i] = inp_vec[i].expr().diff(eps, 1) + nab(inp_vec).contract(xi)[i]
    
    return D_eps_vec

At leading order in the outside-in approach, we have $[\partial_s Y^{a'}] = \dot{z}^a [\partial_s U]$. But since we have constructed our system such that the two are equal it follows that $U = s + \mathcal{O} (\epsilon)$.

In [35]:
U_O1 = s
dU_O1 = U_O1.diff(s, 1)
U_O1, dU_O1

(s, 1)

In [36]:
Deps_dsY = D_eps(dsY)
Deps_dsY[:]

[-(Q*epsilon*q*xi_r(s, epsilon) - Q*q*r)/(m*r^2),
 -((2*Q*epsilon*kappa*q*r^2 - (epsilon^2*kappa*m^2 - kappa)*r^3 + (Q^2*epsilon^2*kappa*q^2 - kappa^3)*r)*xi_ph(s, epsilon) - sqrt(Q^2*epsilon^2*q^2 + 2*Q*epsilon*q*r - (epsilon^2*m^2 - 1)*r^2 - kappa^2)*(Q^2*epsilon*q^2*r - epsilon*m^2*r^3 + Q*q*r^2 - (Q^2*epsilon^2*q^2 + Q*epsilon*q*r - kappa^2)*xi_r(s, epsilon)))/(2*Q*epsilon*m*q*r^3 - (epsilon^2*m^3 - m)*r^4 + (Q^2*epsilon^2*m*q^2 - kappa^2*m)*r^2),
 0,
 (sqrt(Q^2*epsilon^2*q^2 + 2*Q*epsilon*q*r - (epsilon^2*m^2 - 1)*r^2 - kappa^2)*r*xi_ph(s, epsilon) - kappa*xi_r(s, epsilon))/(m*r^3)]

In [37]:
Deps_dsY0 = Deps_dsY.copy()
Deps_dsY0.apply_map(lambda x: x.subs({eps:0}))
Deps_dsY0[:]

[Q*q/(m*r),
 -((kappa^3*r - kappa*r^3)*xi_ph(s, 0) + (Q*q*r^2 + kappa^2*xi_r(s, 0))*sqrt(-kappa^2 + r^2))/(kappa^2*m*r^2 - m*r^4),
 0,
 (sqrt(-kappa^2 + r^2)*r*xi_ph(s, 0) - kappa*xi_r(s, 0))/(m*r^3)]

In [38]:
deps_dsU0 = U.diff(s, 1).diff(eps, 1).subs({eps:0})
dsX1_0 = X1.diff(s, 1).subs({eps:0})
dsX2_0 = X2.diff(s, 1).subs({eps:0})
dsV0 = V.diff(s, 1).subs({eps:0})
deps_dsU0, dsX1_0, dsX2_0

(D[0, 1](U)(s, 0), D[0](X_1)(s, 0), D[0](X_2)(s, 0))

Define the vector field that is the coincidence limit of the $\epsilon$-derivative of the tangent vector, $[D_{\epsilon s} Y^{a'}] = \dot{z}^a [\partial_{\epsilon s} U] + e^a_i [\partial_s X^i]$.

In [39]:
Deps_dsY_CL_out_in = M.vector_field(name=r'D_{\epsilon s} Y_0')
Deps_dsY_CL_out_in = zdot*deps_dsU0 + e1*dsX1_0 + e2*dsX2_0
Deps_dsY_CL_out_in[:]

[d^2(U)/dsd0/m,
 -(kappa*m*d(X_2)/ds - sqrt(kappa + r)*sqrt(-kappa + r)*d^2(U)/dsd0)/(m*r),
 d(X_1)/ds/r,
 (sqrt(kappa + r)*sqrt(-kappa + r)*m*d(X_2)/ds + kappa*d^2(U)/dsd0)/(m*r^2)]

We now compare components of the above two expressions to derive the form of the functions $(U, V, X^1, X^2)$ at the next order. First comparing the $a' = t$ component, we get

In [40]:
O1_t_eq1 = Deps_dsY0[0].expr() == Deps_dsY_CL_out_in[0].expr()
O1_t_eq1

Q*q/(m*r) == D[0, 1](U)(s, 0)/m

To solve this DE, substitute the function $f_U (s) \equiv \partial_{\epsilon} U$, and consequently $\partial_s f_U = \partial_{\epsilon s} U$. Keep in mind that Sage uses the notation $\partial_s U (s, \epsilon) = D_0 (U) (s, \epsilon)$ and $\partial_{\epsilon} U (s, \epsilon) = D_1 (U) (s, \epsilon)$, where the function $U$ takes input $(s, \epsilon)$, i.e. $s$ occupies entry number $0$ in the list $(s, \epsilon)$ and thus $\partial_s \equiv D_0$.

In [41]:
fU = function('f_U')(s)
dfU = fU.diff(s, 1)
fU, dfU

(f_U(s), diff(f_U(s), s))

In [42]:
O1_t_eq2 = O1_t_eq1.lhs().subs({r:r0_sol}) == O1_t_eq1.rhs().subs({deps_dsU0:dfU})
O1_t_eq2

Q*q/(sqrt(kappa^2 + s^2/m^2)*m) == diff(f_U(s), s)/m

In [43]:
desolve(O1_t_eq2, fU, ivar=s)

Q*m*q*arcsinh(s/(kappa*m)) + _C

Thus the first order correction gives $U (s, \epsilon) = s + \epsilon m q Q arcsinh \big (\frac{s}{\kappa m} \big) + \mathcal{O} (\epsilon^2)$. We next compare the $a' = r$ component of the covariant derivative expressions.

In [44]:
U_Oeps = s + eps*m*q*Q*asinh(s/(kap*m))
dsU_Oeps_0 = U_Oeps.diff(s, 1).subs({eps:0})
deps_U_Oeps_0 = U_Oeps.diff(eps, 1).subs({eps:0})
deps_dsU_Oeps = U_Oeps.diff(s, 1).diff(eps, 1).subs({eps:0})
U_Oeps, dsU_Oeps_0, deps_U_Oeps_0, deps_dsU_Oeps

(Q*epsilon*m*q*arcsinh(s/(kappa*m)) + s,
 1,
 Q*m*q*arcsinh(s/(kappa*m)),
 Q*q/(kappa*sqrt(s^2/(kappa^2*m^2) + 1)))

In [45]:
O1_r_eq1 = Deps_dsY0[1].expr() == Deps_dsY_CL_out_in[1].expr()
O1_r_eq1

-((kappa^3*r - kappa*r^3)*xi_ph(s, 0) + (Q*q*r^2 + kappa^2*xi_r(s, 0))*sqrt(-kappa^2 + r^2))/(kappa^2*m*r^2 - m*r^4) == -(kappa*m*D[0](X_2)(s, 0) - sqrt(kappa + r)*sqrt(-kappa + r)*D[0, 1](U)(s, 0))/(m*r)

In [46]:
xi_r0 = xi_r.subs({eps:0})
xi_ph0 = xi_ph.subs({eps:0})
xi_r0, xi_ph0

(xi_r(s, 0), xi_ph(s, 0))

In [47]:
O1_r_eq1_simp_lhs = O1_r_eq1.lhs().subs({xi_r0:xi_CL_out_in[1].expr()}).subs({xi_ph0:xi_CL_out_in[3].expr()})\
.subs({r:r0_sol}).maxima_methods().rootscontract().canonicalize_radical().expand()
O1_r_eq1_simp_lhs

-kappa*m*X_2(s, 0)/(sqrt(kappa^2*m^2 + s^2)*s) + Q*q/s

In [48]:
O1_r_eq1_simp_rhs = O1_r_eq1.rhs().subs({deps_dsU0:deps_dsU_Oeps}).subs({r:r0_sol}).maxima_methods()\
.rootscontract().canonicalize_radical().expand()
O1_r_eq1_simp_rhs

-kappa*m*D[0](X_2)(s, 0)/sqrt(kappa^2*m^2 + s^2) + Q*q*s/(kappa^2*m^2 + s^2)

In [49]:
O1_r_eq2 = O1_r_eq1_simp_lhs == O1_r_eq1_simp_rhs
O1_r_eq2

-kappa*m*X_2(s, 0)/(sqrt(kappa^2*m^2 + s^2)*s) + Q*q/s == -kappa*m*D[0](X_2)(s, 0)/sqrt(kappa^2*m^2 + s^2) + Q*q*s/(kappa^2*m^2 + s^2)

To reduce this to an ODE, introduce the function $f_2 (s) \equiv X^2 (s, \epsilon = 0)$, so that $D_0 (X^2) (s, 0) = \partial_s f_2$.

In [50]:
f2 = function('f_2')(s)
df2 = f2.diff(s, 1)
f2, df2

(f_2(s), diff(f_2(s), s))

In [51]:
O1_r_eq3 = O1_r_eq2.subs({X2.subs({eps:0}):f2}).subs({dsX2_0:df2})
O1_r_eq3

-kappa*m*f_2(s)/(sqrt(kappa^2*m^2 + s^2)*s) + Q*q/s == -kappa*m*diff(f_2(s), s)/sqrt(kappa^2*m^2 + s^2) + Q*q*s/(kappa^2*m^2 + s^2)

In [52]:
f2_sol = desolve(O1_r_eq3, f2, ivar=s)
f2_sol

(_C + sqrt(kappa^2*m^2 + s^2)*Q*q/(kappa*m*s))*s

Thus the leading order form of the coordinate $X^2$ is $X^2 (s, \epsilon) = C_2 s + \frac{qQ}{\kappa m} \sqrt{\kappa^2 m^2 + s^2}$, where the constant $C_2$ is to be determined by choosing appropriate initial conditions for the timelike curves.

In [53]:
C2 = var('C_2')
X2_O1 = C2*s + q*Q*sqrt(kap^2*m^2 + s^2)/(kap*m)
X2_O1

C_2*s + sqrt(kappa^2*m^2 + s^2)*Q*q/(kappa*m)

Next, we compare the $a' = \theta$ component of the covariant derivative expressions.

In [54]:
O1_th_eq1 = Deps_dsY0[2].expr() == Deps_dsY_CL_out_in[2].expr()
O1_th_eq1

0 == D[0](X_1)(s, 0)/r

This clearly implies that $X^1 (s, \epsilon) = C_1 + \mathcal{O} (\epsilon)$, for some constant $C_1$. However, the $\theta$ component of the deviation vector $\xi^{\theta} = \partial_{\epsilon} \theta = 0$ since $\theta = \pi/2$ is fixed. But from the outside-in coincidence limit, this corresponds to $[X^1] = X^1 (s, \epsilon = 0) = 0$.

In [55]:
X1_O1 = 0
X1_O1

0

Note that we have obtained $U$ at $\mathcal{O} (\epsilon)$ and the spatial coordinates $X^i$ at $\mathcal{O} (1)$. This is all the information we can obtain from the first order results (since the first contribution of $V$ only appears at $\mathcal{}O (\epsilon^2)$). Just to check, we will compare the $a' = \phi$ component of the covariant derivative expressions.

In [56]:
O1_ph_eq1 = Deps_dsY0[3].expr() == Deps_dsY_CL_out_in[3].expr()
O1_ph_eq1

(sqrt(-kappa^2 + r^2)*r*xi_ph(s, 0) - kappa*xi_r(s, 0))/(m*r^3) == (sqrt(kappa + r)*sqrt(-kappa + r)*m*D[0](X_2)(s, 0) + kappa*D[0, 1](U)(s, 0))/(m*r^2)

In [57]:
O1_ph_eq1_simp_lhs = O1_ph_eq1.lhs().subs({xi_r0:xi_CL_out_in[1].expr()}).subs({xi_ph0:xi_CL_out_in[3].expr()})\
.subs({r:r0_sol}).maxima_methods().rootscontract().canonicalize_radical().expand()
O1_ph_eq1_simp_lhs

m*X_2(s, 0)/(kappa^2*m^2 + s^2)

In [58]:
O1_ph_eq1_simp_rhs = O1_ph_eq1.rhs().subs({deps_dsU0:deps_dsU_Oeps}).subs({r:r0_sol}).maxima_methods()\
.rootscontract().canonicalize_radical().expand()
O1_ph_eq1_simp_rhs

Q*kappa*m^2*q/(kappa^2*m^2 + s^2)^(3/2) + m*s*D[0](X_2)(s, 0)/(kappa^2*m^2 + s^2)

In [59]:
O1_ph_eq2 = O1_ph_eq1_simp_lhs == O1_ph_eq1_simp_rhs
O1_ph_eq2

m*X_2(s, 0)/(kappa^2*m^2 + s^2) == Q*kappa*m^2*q/(kappa^2*m^2 + s^2)^(3/2) + m*s*D[0](X_2)(s, 0)/(kappa^2*m^2 + s^2)

In [60]:
O1_ph_eq3 = O1_ph_eq2.subs({X2_0:f2}).subs({dsX2_0:df2})
O1_ph_eq3

m*f_2(s)/(kappa^2*m^2 + s^2) == Q*kappa*m^2*q/(kappa^2*m^2 + s^2)^(3/2) + m*s*diff(f_2(s), s)/(kappa^2*m^2 + s^2)

In [61]:
desolve(O1_ph_eq3, f2, ivar=s)

(_C + sqrt(kappa^2*m^2 + s^2)*Q*q/(kappa*m*s))*s

This is exactly what we derived from comparing the $a' = r$ component above.

Thus from the first order results, we find that
$U = s + \epsilon m q Q arcsinh \big ( \frac{s}{\kappa m} \big ), \\
X^1 = 0, \\
X^2 = C_2 s + \frac{qQ}{\kappa m} \sqrt{\kappa^2 m^2 + s^2}$.

To derive the form of $V$, rather than using the second order derivative of the tangent vector, we instead consider the leading order expression for the line element in the outside-in approach. We have

$\partial_s Y^{a'} \partial_s Y_{a'} = \epsilon^2 \big ( -2 [\partial_s U \partial_s V] + [\partial_s X^i \partial_s X_i] \big )$

In [62]:
dsY2 = g(dsY,dsY)
dsY2.expr()

-epsilon^2

In [63]:
line_el_out_in_LO = -2*dsU0*dsV0 + dsX1_0*dsX1_0 + dsX2_0*dsX2_0
line_el_out_in_LO

-2*D[0](U)(s, 0)*D[0](V)(s, 0) + D[0](X_1)(s, 0)^2 + D[0](X_2)(s, 0)^2

In [64]:
Oeps2_line_el = dsY2.expr() == eps^2*line_el_out_in_LO
Oeps2_line_el

-epsilon^2 == -(2*D[0](U)(s, 0)*D[0](V)(s, 0) - D[0](X_1)(s, 0)^2 - D[0](X_2)(s, 0)^2)*epsilon^2

In [65]:
Oeps2_line_el_2 = (Oeps2_line_el/(eps^2)).subs({dsU0:dsU_Oeps_0}).subs({dsX1_0:0})\
.subs({dsX2_0:X2_O1.diff(s, 1)})
Oeps2_line_el_2

-1 == (C_2 + Q*q*s/(sqrt(kappa^2*m^2 + s^2)*kappa*m))^2 - 2*D[0](V)(s, 0)

In [66]:
Oeps2_line_el_2.rhs().operands()[:]

[(C_2 + Q*q*s/(sqrt(kappa^2*m^2 + s^2)*kappa*m))^2, -2*D[0](V)(s, 0)]

In [67]:
Oeps2_line_el_3 = (Oeps2_line_el_2 - Oeps2_line_el_2.lhs() - Oeps2_line_el_2.rhs().operands()[1])/2
Oeps2_line_el_3

D[0](V)(s, 0) == 1/2*(C_2 + Q*q*s/(sqrt(kappa^2*m^2 + s^2)*kappa*m))^2 + 1/2

In [68]:
fv = function('f_v')(s)
dfv = fv.diff(s, 1)
fv, dfv

(f_v(s), diff(f_v(s), s))

In [69]:
Oeps2_line_el_4 = Oeps2_line_el_3.subs({dsV0:dfv})
Oeps2_line_el_4

diff(f_v(s), s) == 1/2*(C_2 + Q*q*s/(sqrt(kappa^2*m^2 + s^2)*kappa*m))^2 + 1/2

In [70]:
V0_O1 = desolve(Oeps2_line_el_4, fv, ivar=s).simplify_full().expand().collect(s)
V0_O1

-1/2*Q^2*q^2*arctan(s/(kappa*m))/(kappa*m) + 1/2*(C_2^2 + Q^2*q^2/(kappa^2*m^2) + 1)*s + sqrt(kappa^2*m^2 + s^2)*C_2*Q*q/(kappa*m) + _C

We would now like to plug these reults into the expression for the coincidence limit of the deviation vector in order to derive the correction to the original coordinates.

In [71]:
Oeps_t_corr1 = xi_CL_out_in[0].expr()
Oeps_t_corr1

D[1](U)(s, 0)/m

In [72]:
Oeps_t_corr2 = Oeps_t_corr1.subs({depsU0:deps_U_Oeps_0})
Oeps_t_corr2

Q*q*arcsinh(s/(kappa*m))

In [73]:
Oeps_r_corr1 = xi_CL_out_in[1].expr()
Oeps_r_corr1

-(kappa*m*X_2(s, 0) - sqrt(kappa + r)*sqrt(-kappa + r)*D[1](U)(s, 0))/(m*r)

In [74]:
Oeps_r_corr2 = Oeps_r_corr1.subs({X2_0:X2_O1}).subs({depsU0:deps_U_Oeps_0}).subs({r:r0_sol}).maxima_methods()\
.rootscontract().canonicalize_radical().expand()
Oeps_r_corr2

-C_2*kappa*m*s/sqrt(kappa^2*m^2 + s^2) + Q*q*s*arcsinh(s/(kappa*m))/sqrt(kappa^2*m^2 + s^2) - Q*q

In [75]:
Oeps_ph_corr1 = xi_CL_out_in[3].expr()
Oeps_ph_corr1

(sqrt(kappa + r)*sqrt(-kappa + r)*m*X_2(s, 0) + kappa*D[1](U)(s, 0))/(m*r^2)

In [76]:
Oeps_ph_corr2 = Oeps_ph_corr1.subs({X2_0:X2_O1}).subs({depsU0:deps_U_Oeps_0}).subs({r:r0_sol})\
.maxima_methods().rootscontract().canonicalize_radical().expand()
Oeps_ph_corr2

Q*kappa^2*m^2*q*arcsinh(s/(kappa*m))/(kappa^3*m^2 + kappa*s^2) + C_2*kappa*m*s^2/(kappa^3*m^2 + kappa*s^2) + sqrt(kappa^2*m^2 + s^2)*Q*q*s/(kappa^3*m^2 + kappa*s^2)

In [77]:
Oeps_ph_corr2.operands()[:]

[Q*kappa^2*m^2*q*arcsinh(s/(kappa*m))/(kappa^3*m^2 + kappa*s^2),
 C_2*kappa*m*s^2/(kappa^3*m^2 + kappa*s^2),
 sqrt(kappa^2*m^2 + s^2)*Q*q*s/(kappa^3*m^2 + kappa*s^2)]

In [78]:
Oeps_ph_corr2.operands()[0].simplify_full(), Oeps_ph_corr2.operands()[1].simplify_full(),\
Oeps_ph_corr2.operands()[2].factor()

(Q*kappa*m^2*q*arcsinh(s/(kappa*m))/(kappa^2*m^2 + s^2),
 C_2*m*s^2/(kappa^2*m^2 + s^2),
 Q*q*s/(sqrt(kappa^2*m^2 + s^2)*kappa))

In [79]:
Oeps_ph_corr3 = Oeps_ph_corr2\
.subs({Oeps_ph_corr2.operands()[0]:Oeps_ph_corr2.operands()[0].simplify_full()})\
.subs({Oeps_ph_corr2.operands()[1]:Oeps_ph_corr2.operands()[1].simplify_full()})\
.subs({Oeps_ph_corr2.operands()[2]:Oeps_ph_corr2.operands()[2].factor()})
Oeps_ph_corr3

Q*kappa*m^2*q*arcsinh(s/(kappa*m))/(kappa^2*m^2 + s^2) + C_2*m*s^2/(kappa^2*m^2 + s^2) + Q*q*s/(sqrt(kappa^2*m^2 + s^2)*kappa)

Thus the first order corrections to the original coordinates $(t, r, \theta, \phi)$ are

$
t (s, \epsilon) = \frac{s}{m} + \epsilon q Q arcsinh \big ( \frac{s}{\kappa m} \big ) + \mathcal{O} (\epsilon^2), \\
r (s, \epsilon) = \sqrt{\kappa^2 + \frac{s^2}{m^2}} + \epsilon \bigg \{ - qQ - \frac{C_2 \kappa m s}{\sqrt{\kappa^2 m^2 + s^2}} + \frac{q Q s}{\sqrt{\kappa^2 m^2 + s^2}} arcsinh{ \big ( \frac{s}{\kappa m} \big ) } \bigg \} + \mathcal{O} (\epsilon^2), \\
\theta (s, \epsilon) = \frac{\pi}{2}, \\
\phi (s, \epsilon) = \Phi_0 + \arctan{\big ( \frac{s}{\kappa m} \big )} + \epsilon \bigg \{ \frac{q Q s}{\kappa \sqrt{\kappa^2 m^2 + s^2}} + \frac{C_2 m s^2}{\kappa^2 m^2 + s^2} + \frac{q Q \kappa m^2}{\kappa^2 m^2 + s^2} arcsinh{ \big ( \frac{s}{\kappa m} \big ) } \bigg \} + \mathcal{O} (\epsilon^2)
$

In [80]:
t_Oeps = t0_sol + eps*Oeps_t_corr2
r_Oeps = r0_sol + eps*Oeps_r_corr2
ph_Oeps = ph0_sol + eps*Oeps_ph_corr3
t_Oeps, r_Oeps, ph_Oeps

(Q*epsilon*q*arcsinh(s/(kappa*m)) + s/m,
 -(C_2*kappa*m*s/sqrt(kappa^2*m^2 + s^2) - Q*q*s*arcsinh(s/(kappa*m))/sqrt(kappa^2*m^2 + s^2) + Q*q)*epsilon + sqrt(kappa^2 + s^2/m^2),
 (Q*kappa*m^2*q*arcsinh(s/(kappa*m))/(kappa^2*m^2 + s^2) + C_2*m*s^2/(kappa^2*m^2 + s^2) + Q*q*s/(sqrt(kappa^2*m^2 + s^2)*kappa))*epsilon + Phi_0 + arctan(s/(kappa*m)))

In [81]:
dt_Oeps = t_Oeps.diff(s, 1)
dr_Oeps = r_Oeps.diff(s, 1)
dph_Oeps = ph_Oeps.diff(s, 1)
dt_Oeps, dr_Oeps, dph_Oeps

(Q*epsilon*q/(kappa*m*sqrt(s^2/(kappa^2*m^2) + 1)) + 1/m,
 (C_2*kappa*m*s^2/(kappa^2*m^2 + s^2)^(3/2) - Q*q*s^2*arcsinh(s/(kappa*m))/(kappa^2*m^2 + s^2)^(3/2) - C_2*kappa*m/sqrt(kappa^2*m^2 + s^2) + Q*q*arcsinh(s/(kappa*m))/sqrt(kappa^2*m^2 + s^2) + Q*q*s/(sqrt(kappa^2*m^2 + s^2)*kappa*m*sqrt(s^2/(kappa^2*m^2) + 1)))*epsilon + s/(sqrt(kappa^2 + s^2/m^2)*m^2),
 -(2*Q*kappa*m^2*q*s*arcsinh(s/(kappa*m))/(kappa^2*m^2 + s^2)^2 + 2*C_2*m*s^3/(kappa^2*m^2 + s^2)^2 - 2*C_2*m*s/(kappa^2*m^2 + s^2) + Q*q*s^2/((kappa^2*m^2 + s^2)^(3/2)*kappa) - Q*m*q/((kappa^2*m^2 + s^2)*sqrt(s^2/(kappa^2*m^2) + 1)) - Q*q/(sqrt(kappa^2*m^2 + s^2)*kappa))*epsilon + 1/(kappa*m*(s^2/(kappa^2*m^2) + 1)))

Define a function to compute the covariant $s$-derivative of a vector, and check if these curves have $0$ acceleration when the test particles are uncharged, i.e. $q = 0$.

In [85]:
def D_s(inp_vec):
    D_s_vec = M.vector_field(name=r'D_{s}V')
    for i in range(4):
        D_s_vec[i] = nab(inp_vec).contract(dsY)[i]
    
    return D_s_vec

In [86]:
Ds2Y = nab(dsY).contract(dsY)
Ds2Y[:]

[-sqrt(Q^2*epsilon^2*q^2 + 2*Q*epsilon*q*r - (epsilon^2*m^2 - 1)*r^2 - kappa^2)*Q*epsilon*q/(m^2*r^3),
 -(Q^2*epsilon^2*q^2 + Q*epsilon*q*r)/(m^2*r^3),
 0,
 0]

In [87]:
Deps_Ds2Y = D_eps(Ds2Y)
Deps_Ds2Y[:]

[-(3*Q^2*epsilon*q^2*r^2 - (2*Q*epsilon^2*m^2 - Q)*q*r^3 + (2*Q^3*epsilon^2*q^3 - Q*kappa^2*q)*r - (3*Q^3*epsilon^3*q^3 + 5*Q^2*epsilon^2*q^2*r - 3*Q*epsilon*kappa^2*q - 2*(Q*epsilon^3*m^2 - Q*epsilon)*q*r^2)*xi_r(s, epsilon))*sqrt(Q^2*epsilon^2*q^2 + 2*Q*epsilon*q*r - (epsilon^2*m^2 - 1)*r^2 - kappa^2)/(2*Q*epsilon*m^2*q*r^5 - (epsilon^2*m^4 - m^2)*r^6 + (Q^2*epsilon^2*m^2*q^2 - kappa^2*m^2)*r^4),
 -(2*Q^2*epsilon*q^2*r + Q*q*r^2 - (3*Q^2*epsilon^2*q^2 + 2*Q*epsilon*q*r)*xi_r(s, epsilon))/(m^2*r^4),
 0,
 -(Q^2*epsilon^2*q^2 + Q*epsilon*q*r)*xi_ph(s, epsilon)/(m^2*r^4)]

In [88]:
term_t_q0_1 = Deps_Ds2Y[0].expr()
term_t_q0_1

-(3*Q^2*epsilon*q^2*r^2 - (2*Q*epsilon^2*m^2 - Q)*q*r^3 + (2*Q^3*epsilon^2*q^3 - Q*kappa^2*q)*r - (3*Q^3*epsilon^3*q^3 + 5*Q^2*epsilon^2*q^2*r - 3*Q*epsilon*kappa^2*q - 2*(Q*epsilon^3*m^2 - Q*epsilon)*q*r^2)*xi_r(s, epsilon))*sqrt(Q^2*epsilon^2*q^2 + 2*Q*epsilon*q*r - (epsilon^2*m^2 - 1)*r^2 - kappa^2)/(2*Q*epsilon*m^2*q*r^5 - (epsilon^2*m^4 - m^2)*r^6 + (Q^2*epsilon^2*m^2*q^2 - kappa^2*m^2)*r^4)

In [89]:
term_t_q0_2 = term_t_q0_1.subs({r:r_Oeps}).subs({q:0})
term_t_q0_2

0

In [90]:
term_r_q0_1 = Deps_Ds2Y[1].expr()
term_r_q0_1

-(2*Q^2*epsilon*q^2*r + Q*q*r^2 - (3*Q^2*epsilon^2*q^2 + 2*Q*epsilon*q*r)*xi_r(s, epsilon))/(m^2*r^4)

In [91]:
term_r_q0_2 = term_r_q0_1.subs({r:r_Oeps}).subs({q:0})
term_r_q0_2

0

In [92]:
term_ph_q0_1 = Deps_Ds2Y[1].expr()
term_ph_q0_1

-(2*Q^2*epsilon*q^2*r + Q*q*r^2 - (3*Q^2*epsilon^2*q^2 + 2*Q*epsilon*q*r)*xi_r(s, epsilon))/(m^2*r^4)

In [93]:
term_ph_q0_2 = term_ph_q0_1.subs({r:r_Oeps})
term_ph_q0_2.subs({q:0})

0

In [94]:
ds2t = t_Oeps.diff(s, 2)
ds2t.simplify_full()

-Q*epsilon*q*s/(kappa^2*m^2 + s^2)^(3/2)

In [95]:
Ds2_r = r_Oeps.diff(s, 2) - (kap^2/(m^2*r^3)).subs({r:r_Oeps})
Ds2_r

-(3*C_2*kappa*m*s^3/(kappa^2*m^2 + s^2)^(5/2) - 3*Q*q*s^3*arcsinh(s/(kappa*m))/(kappa^2*m^2 + s^2)^(5/2) - 3*C_2*kappa*m*s/(kappa^2*m^2 + s^2)^(3/2) + 3*Q*q*s*arcsinh(s/(kappa*m))/(kappa^2*m^2 + s^2)^(3/2) + 2*Q*q*s^2/((kappa^2*m^2 + s^2)^(3/2)*kappa*m*sqrt(s^2/(kappa^2*m^2) + 1)) - 2*Q*q/(sqrt(kappa^2*m^2 + s^2)*kappa*m*sqrt(s^2/(kappa^2*m^2) + 1)) + Q*q*s^2/(sqrt(kappa^2*m^2 + s^2)*kappa^3*m^3*(s^2/(kappa^2*m^2) + 1)^(3/2)))*epsilon + 1/(sqrt(kappa^2 + s^2/m^2)*m^2) + kappa^2/(((C_2*kappa*m*s/sqrt(kappa^2*m^2 + s^2) - Q*q*s*arcsinh(s/(kappa*m))/sqrt(kappa^2*m^2 + s^2) + Q*q)*epsilon - sqrt(kappa^2 + s^2/m^2))^3*m^2) - s^2/((kappa^2 + s^2/m^2)^(3/2)*m^4)

In [96]:
Ds2_ph = ph_Oeps.diff(s, 2) + ((2/r)*ds_ph*ds_r).subs({r:r_Oeps})
Ds2_ph

(8*Q*kappa*m^2*q*s^2*arcsinh(s/(kappa*m))/(kappa^2*m^2 + s^2)^3 - 2*Q*kappa*m^2*q*arcsinh(s/(kappa*m))/(kappa^2*m^2 + s^2)^2 + 8*C_2*m*s^4/(kappa^2*m^2 + s^2)^3 - 10*C_2*m*s^2/(kappa^2*m^2 + s^2)^2 + 3*Q*q*s^3/((kappa^2*m^2 + s^2)^(5/2)*kappa) - 4*Q*m*q*s/((kappa^2*m^2 + s^2)^2*sqrt(s^2/(kappa^2*m^2) + 1)) + 2*C_2*m/(kappa^2*m^2 + s^2) - 3*Q*q*s/((kappa^2*m^2 + s^2)^(3/2)*kappa) - Q*q*s/((kappa^2*m^2 + s^2)*kappa^2*m*(s^2/(kappa^2*m^2) + 1)^(3/2)))*epsilon - 2*sqrt(-(m^2 - Q^2*q^2/((C_2*kappa*m*s/sqrt(kappa^2*m^2 + s^2) - Q*q*s*arcsinh(s/(kappa*m))/sqrt(kappa^2*m^2 + s^2) + Q*q)*epsilon - sqrt(kappa^2 + s^2/m^2))^2)*epsilon^2 - 2*Q*epsilon*q/((C_2*kappa*m*s/sqrt(kappa^2*m^2 + s^2) - Q*q*s*arcsinh(s/(kappa*m))/sqrt(kappa^2*m^2 + s^2) + Q*q)*epsilon - sqrt(kappa^2 + s^2/m^2)) - kappa^2/((C_2*kappa*m*s/sqrt(kappa^2*m^2 + s^2) - Q*q*s*arcsinh(s/(kappa*m))/sqrt(kappa^2*m^2 + s^2) + Q*q)*epsilon - sqrt(kappa^2 + s^2/m^2))^2 + 1)*kappa/(((C_2*kappa*m*s/sqrt(kappa^2*m^2 + s^2) - Q*q*s*arcsinh(s/(kappa*m))/sqrt(kappa^2*m^2 + s^2) + Q*q)*epsilon - sqrt(kappa^2 + s^2/m^2))^3*m^2) - 2*s/(kappa^3*m^3*(s^2/(kappa^2*m^2) + 1)^2)

In [97]:
Ds2_ph.subs({eps:0}).simplify_full()

0

In [98]:
Ds2_r.diff(eps, 1).subs({eps:0}).subs({q:0})

-3*C_2*kappa*m*s^3/(kappa^2*m^2 + s^2)^(5/2) + 3*C_2*kappa*m*s/(kappa^2*m^2 + s^2)^(3/2) - 3*C_2*kappa^3*s/(sqrt(kappa^2*m^2 + s^2)*(kappa^2 + s^2/m^2)^2*m)

In [99]:
Ds2_ph.diff(eps, 1).subs({eps:0}).subs({q:0}).simplify_full()

0

Clearly the relevant components of the 4-acceleration vanish, implying that the curves are geodesic when $q = 0$, for all values of $C_2$. But this constant can be used to fix the $s$ value at which the timelike curves attain the minimum radius.

In [100]:
dsr_Oeps = r_Oeps.diff(s, 1)
dsr_Oeps

(C_2*kappa*m*s^2/(kappa^2*m^2 + s^2)^(3/2) - Q*q*s^2*arcsinh(s/(kappa*m))/(kappa^2*m^2 + s^2)^(3/2) - C_2*kappa*m/sqrt(kappa^2*m^2 + s^2) + Q*q*arcsinh(s/(kappa*m))/sqrt(kappa^2*m^2 + s^2) + Q*q*s/(sqrt(kappa^2*m^2 + s^2)*kappa*m*sqrt(s^2/(kappa^2*m^2) + 1)))*epsilon + s/(sqrt(kappa^2 + s^2/m^2)*m^2)

In [101]:
rmin_eq1 = dsr_Oeps == 0
rmin_eq2 = rmin_eq1.factor()
rmin_eq3 = rmin_eq2*(rmin_eq2.lhs().denominator())
rmin_eq3.factor()

-C_2*epsilon*kappa^3*m^4 + Q*epsilon*kappa^2*m^3*q*arcsinh(s/(kappa*m)) + sqrt(kappa^2*m^2 + s^2)*Q*epsilon*m*q*s + kappa^2*m^2*s + s^3 == 0

In [117]:
rmin_eq4 = rmin_eq3 - rmin_eq3.lhs().operands()[0]
rmin_eq5 = (rmin_eq4/(eps*kap^3*m^4)).expand()
rmin_eq5

Q*q*arcsinh(s/(kappa*m))/(kappa*m) + sqrt(kappa^2*m^2 + s^2)*Q*q*s/(kappa^3*m^3) + s/(epsilon*kappa*m^2) + s^3/(epsilon*kappa^3*m^4) == C_2

The leading order solution $r_0 = \sqrt{\kappa^2 + s^2/m^2}$ has a minima at $s = 0$, corresponding to $r_0 = \kappa$. Thus we assume that the timelike curves attain their minimal $r$ value close to $s = 0$. For simplicity, let us assume $s \sim \mathcal{O} (\epsilon)$, and Taylor expand the above LHS around $s = 0$. Since $s$ is close to $0$, we drop terms of order $\mathcal{O} (s^3)$.

In [123]:
rmin_eq6 = rmin_eq5.lhs().taylor(s, 0, 2) == rmin_eq5.rhs()
rmin_eq6

(2*Q*epsilon*q + kappa)*s/(epsilon*kappa^2*m^2) == C_2

In [125]:
rmin_eq7 = rmin_eq6*rmin_eq6.lhs().denominator()/(2*eps*q*Q + kap)
rmin_eq7

s == C_2*epsilon*kappa^2*m^2/(2*Q*epsilon*q + kappa)

Thus at leading order, the timelike curves attain their minimal radial value at $s = \frac{\epsilon C_2 \kappa^2 m^2}{\kappa + 2 \epsilon q Q}$.

In [126]:
smin_Oeps = eps*C2*kap^2*m^2/(kap + 2*eps*q*Q)
smin_Oeps

C_2*epsilon*kappa^2*m^2/(2*Q*epsilon*q + kappa)

In [129]:
rmin_val = (r_Oeps(s=smin_Oeps)).taylor(eps, 0, 2)
rmin_val

-1/2*C_2^2*epsilon^2*kappa*m^2 - Q*epsilon*q + kappa

Thus the leading order minima of the radial function for the timelike curves is $r_{\text{min}} = \kappa - \epsilon q Q - \frac{1}{2} C_2^2 \epsilon^2 \kappa m^2$.

We would like to check if this is also the $s$-value at which the angle $\phi$ is minimised.

In [130]:
dsph_Oeps = ph_Oeps.diff(s, 1)
dsph_Oeps

-(2*Q*kappa*m^2*q*s*arcsinh(s/(kappa*m))/(kappa^2*m^2 + s^2)^2 + 2*C_2*m*s^3/(kappa^2*m^2 + s^2)^2 - 2*C_2*m*s/(kappa^2*m^2 + s^2) + Q*q*s^2/((kappa^2*m^2 + s^2)^(3/2)*kappa) - Q*m*q/((kappa^2*m^2 + s^2)*sqrt(s^2/(kappa^2*m^2) + 1)) - Q*q/(sqrt(kappa^2*m^2 + s^2)*kappa))*epsilon + 1/(kappa*m*(s^2/(kappa^2*m^2) + 1))

In [131]:
phmin_eq1 = (dsph_Oeps).factor() == 0
phmin_eq2 = phmin_eq1*phmin_eq1.lhs().denominator()/(kap*m)
phmin_eq2

2*Q*epsilon*kappa^2*m^3*q + 2*sqrt(kappa^2*m^2 + s^2)*C_2*epsilon*kappa*m^2*s - 2*sqrt(kappa^2*m^2 + s^2)*Q*epsilon*m*q*s*arcsinh(s/(kappa*m)) + 2*Q*epsilon*m*q*s^2 + sqrt(kappa^2*m^2 + s^2)*kappa^2*m^2 + sqrt(kappa^2*m^2 + s^2)*s^2 == 0

In [134]:
phmin_eq3 = phmin_eq2.lhs().taylor(s, 0, 3).collect(C2) == phmin_eq2.rhs()
phmin_eq3

2*Q*epsilon*kappa^2*m^3*q + kappa^3*m^3 + 3/2*kappa*m*s^2 + (2*epsilon*kappa^2*m^3*s + epsilon*m*s^3)*C_2 == 0

In [135]:
phmin_eq3.lhs().operands()[:]

[2*Q*epsilon*kappa^2*m^3*q,
 kappa^3*m^3,
 3/2*kappa*m*s^2,
 (2*epsilon*kappa^2*m^3*s + epsilon*m*s^3)*C_2]

In [136]:
phmin_eq4 = phmin_eq3 - phmin_eq3.lhs().operands()[3]
phmin_eq4

2*Q*epsilon*kappa^2*m^3*q + kappa^3*m^3 + 3/2*kappa*m*s^2 == -(2*epsilon*kappa^2*m^3*s + epsilon*m*s^3)*C_2

In [139]:
dsph_Oeps(s = smin_Oeps).taylor(eps, 0, 3)

-5*C_2^2*Q*epsilon^3*m*q/kappa^2 + C_2^2*epsilon^2*m/kappa + 2*Q*epsilon*q/(kappa^2*m) + 1/(kappa*m)

In [142]:
ph_Oeps(s = smin_Oeps).taylor(eps, 0, 3)

2/3*C_2^3*epsilon^3*m^3 + C_2*epsilon*m + Phi_0

In [143]:
ph_Oeps

(Q*kappa*m^2*q*arcsinh(s/(kappa*m))/(kappa^2*m^2 + s^2) + C_2*m*s^2/(kappa^2*m^2 + s^2) + Q*q*s/(sqrt(kappa^2*m^2 + s^2)*kappa))*epsilon + Phi_0 + arctan(s/(kappa*m))